# 5. 이미지 처리 능력이 탁월한 CNN

## 5.0. 요약

### 5.0.1. 지난장

지난 4장에서는 3장의 ANN에서 다수의 은닉층을 활용한 DNN을 생성하여 패션아이템을 구분하는 딥러닝 모델을 만들었다. 그리고, 데이터의 과적합을 방지하고 일반화 성능을 높이기 위해 사용되는 pytorch의 argumentation tool인 transform과 모델에서 학습을 수행하는 뉴런을 컨트롤하여 과적합을 피하는 드롭아웃에 대해서 살펴보았다.

### 5.0.2. 이번장

이번 5장에서는 이미지 분류나 특정 패턴등과 같은 유사한 형태의 무언가를 찾을 때 많이 활용되는 CNN(Convolutional Neural Network)에 대해서 유래와 이를 간단하게 적용한 모델에 대해 알아보자. 그리고 CNN 모델의 한계를 극복하기 위해서 새로 고안된 Neural Net 중 성능이 좋다고 알려진 ResNet에 대해서 알아보고 구현해보자.

## 5.1 CNN 기초

### CNN(Convolutional Neural Network)

CNN은 고양이가 이미지를 어떻게 인식하는지에 대한 실험을 통해 고안된 알고리즘이다. 고양이가 이미지를 인식할 때, 이미지의 특정 부분을 보고 인지할 때마다 뇌에서 반응하는 영역이 달랐다. 그러므로, 이미지를 인식할 때는 전체를 한 번에 인식하는 것이 아니라 부분으로 나누어서 인지한 다음에 이를 모아서 특정 객체를 인지하는 것이다.

이는 사람의 경우에도 마찬가지이다. 특정 사람을 인지하고 구분하기 위해서는 얼굴의 눈, 코, 입 등의 특징을 파악하여 종합한 뒤에 특정인을 구분하기 때문이다.

이 접근법을 활용하여 특정 오브젝트의 부분을 인지하여 전체를 인지하는 것을 토대로 만들어진 신경망 모델이 합성곱신경망(CNN, Convolutional Network)이다. 이는 이미지나 비디오 같은 영상 인식에 특화된 설계로 병렬처리가 쉬워 대규모 서비스에 적용할 수 있으며, 최근에는 이미지 뿐만 아니라 자연어 처리등의 사례에서 사용된다.

### 5.1. 컴퓨터가 보는 이미지

우리가 앞선 4장에서 생성한 DNN모델은 하나의 패션 아이템 28x28 픽셀의 이미지를 1x784로 변환하여 전체적으로 한 번 학습했다고 할 수 있다.

하지만, 이는 주어지는 이미지가 한쪽에 치우치는 등의 노이즈가 있게되면 유의미하게 구분할 수 없게된다. 그렇다고 치우친 정도에 따라서 나누어서 학습하는 것 또한 이상적이지 못하다. 그러므로 우리는 이 문제에 대응할 수 있도록 부분으로 나누어 학습하는 Convolutional한 방식의 학습이 필요하다.

### 5.2. 컨볼루션

CNN에서 우리는 하나의 이미지를 여러개의 구역으로 나누고 그 구역에서 특징을 대표할 수 있는 값으로 치환하여 추출하는 필터를 활용한다. 이 필터를 적용함에 있어 이미지를 왼쪽 위에서 오른쪽 밑 까지 밀어가면서 곱하고 더하는데 이를 컨볼루션이라고 한다.

과거에는 이 필터를 수작업으로 선언하고 디자인하였지만, CNN 모델의 목적은 이 필터를 학습하는 것이며 하나의 필터는 하나의 신경망이 된느 것이다.

### 5.3. CNN 모델

CNN모델은 보통 컨볼루션 계층, 풀링계층, 특징들을 모아 최종 분류하는 인공 신경망 계층으로 구분된다. 풀링 계층에서 컨볼루션 계층을 통해 얻어진 feature map에서 가장 중요한 특징을 걸러냄으로써 차원이 축소된다.

* 컨볼루션 계층: 이미지의 특징을 추출
* 풀링 계층: 필터를 거친 여러 특징 중 가장 중요한 특징을 골라내는 역할

#### 컨볼루션 계층 주요 파라미터

* 컨볼루션 필터 or 커널(kernel): feature map으로 특징을 추출하는데 있어서 이미지를 나누는 단위
* 스트라이드(Stride): 커널을 움직이는 단위

컨볼루션 계층을 통해 생성된 feature map의 크기가 클 수록 학습이 어렵고 과적합의 유험이 있으므로, 필터의 크기를 적절하게 만들어 주어야 한다. 

그리고 feature map을 가지고 풀링 계층의 연산을 수행하며, 풀링 계층 또한 일종의 컨볼루션 연산을 수행하는 계층이다. 여기서는 커널 필터와 같이 일정 크기의 픽셀을 묶어서 평균 / 최댓값 등의 간단한 연산을 수행한다.

## 5.2. CNN 모델 구현하기
해당 절에서 구현할 CNN 모델은 2개의 Convolutional, Pooling layer를 거치고 인공신경망을 통해 분류하는 간단한 CNN 모델입니다.

In [5]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

from tqdm import tqdm_notebook

from copy import deepcopy

In [6]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(USE_CUDA,DEVICE)

False cpu


In [7]:
EPOCHS = 40
BATCH_SIZE = 64

### transforms.Normalize

Data Agumentation 수행 시, 왜 Normalize를 거치는 지 알고 있는가? 해당 메서드는 입력으로 들어온 이미지 벡터들에 대해서, 특정 수준으로 정규화를 수행하는 메서드이다. 정규화를 하게 되면, CNN의 학습속도를 더 빠르게 해주고 첨도(skewness)또한 줄여주므로 성능향상에 도움이 되기 때문이라고 한다.

link. https://discuss.pytorch.org/t/understanding-transform-normalize/21730

In [8]:
transform = transforms.Compose([
#     transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [9]:
train_dataset = datasets.FashionMNIST(
    root = './data/',
    train = True,
#     download = True,
    download = False,
    transform = transform
)

test_dataset = datasets.FashionMNIST(
    root = './data/',
    train = False,
#     download = True,
    download = False,
    transform = transform
)

In [10]:
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False
)

### CNN 모델 생성

이제 앞서 언급한 형태의 CNN 모델을 선언하여 생성해보자

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5,stride=1)
        self.conv2 = nn.Conv2d(10,20, kernel_size=5,stride=1)
        
        self.drop = nn.Dropout2d()
        
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    def forward(self, x):
        x= F.relu(F.max_pool2d(self.conv1(x),2))
        x= F.relu(F.max_pool2d(self.conv2(x),2))
        
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [12]:
model = CNN().to(DEVICE) if USE_CUDA else CNN()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [13]:
model

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [14]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [15]:
def train(model, train_loader, optimizer, epoch, batch_size):
    best_loss = np.inf
    best_batch_num = 0
    best_accuarcy = np.inf
    best_state_dict = deepcopy(model.state_dict())
    
    model.train()
    
    for batch_num, (inputs, labels) in enumerate(tqdm_notebook(train_loader)):
        if USE_CUDA:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
        
        model.zero_grad()
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if batch_num % 200 == 0:
            output_labels = outputs.argmax(axis=1)
            accuarcy = output_labels.eq(labels.view_as(output_labels)).sum().item() / batch_size
        
            print("epoch: {} | batch_num: {}; loss: {}; accuracy: {}".format(epoch, batch_num, loss.item(), accuarcy))
            
        if loss < best_loss:
            best_loss = loss
            best_state_dict = deepcopy(model.state_dict())
            best_batch_num = batch_num
            output_labels = outputs.argmax(axis=1)
            best_accuracy = output_labels.eq(labels.view_as(output_labels)).sum().item() / batch_size
#             print("state saved at {} batch_num of loss: {}".format(batch_num,loss))
            
    print("epoch: {} | Minimum loss of model during training is batch_num: {}; loss: {}; accuarcy: {}".format(epoch, best_batch_num, best_loss, best_accuracy))
    model.load_state_dict(best_state_dict)
    return model

In [16]:
def evaluate(model,test_loader, epoch, batch_size):
    batch_num = len(test_loader)
    len_dataset = len(test_loader.dataset)
    acc_correct_labels = 0
    acc_loss = 0.0
    
    model.eval()
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            if USE_CUDA:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                
            outputs = model(inputs)
            loss = F.cross_entropy(outputs,labels)
            acc_loss += loss 
            
            output_labels = outputs.argmax(axis=1)
            correct_labels = output_labels.eq(labels.view_as(output_labels)).sum().item()
            acc_correct_labels += correct_labels
            
    acc_loss = acc_loss / batch_num
    acc_accuracy = acc_correct_labels / len_dataset
    print("epoch: {} | test loss: {}; accuarcy: {}".format(epoch, acc_loss, acc_accuracy))

In [13]:
for epoch in range(EPOCHS):
    
    print("="*20,"epoch {}".format(epoch))
    model = train(model, train_loader, optimizer, epoch, BATCH_SIZE)
    evaluate(model, test_loader, epoch, BATCH_SIZE)
    
    if epoch > 4:
        break

==================== epoch 0


epoch: 0 | batch_num: 0; loss: 2.3125736713409424; accuracy: 0.109375
epoch: 0 | batch_num: 200; loss: 1.1004817485809326; accuracy: 0.484375
epoch: 0 | batch_num: 400; loss: 0.9184608459472656; accuracy: 0.625
epoch: 0 | batch_num: 600; loss: 0.9456873536109924; accuracy: 0.65625
epoch: 0 | batch_num: 800; loss: 0.48903217911720276; accuracy: 0.828125

epoch: 0 | Minimum loss of model during training is batch_num: 712; loss: 0.4885271489620209; accuarcy: 0.796875
epoch: 0 | test loss: 0.6537640690803528; accuarcy: 0.7445
==================== epoch 1


epoch: 1 | batch_num: 0; loss: 0.8017565608024597; accuracy: 0.65625
epoch: 1 | batch_num: 200; loss: 0.8350003957748413; accuracy: 0.734375
epoch: 1 | batch_num: 400; loss: 0.7882067561149597; accuracy: 0.734375
epoch: 1 | batch_num: 600; loss: 0.754618227481842; accuracy: 0.734375
epoch: 1 | batch_num: 800; loss: 0.6779388785362244; accuracy: 0.703125

epoch: 1 | Minimum loss of model during training is batch_num: 465; loss: 0.37547773122787476; accuarcy: 0.859375
epoch: 1 | test loss: 0.5961498618125916; accuarcy: 0.7653
==================== epoch 2


epoch: 2 | batch_num: 0; loss: 0.7119736075401306; accuracy: 0.703125
epoch: 2 | batch_num: 200; loss: 0.6543973088264465; accuracy: 0.71875
epoch: 2 | batch_num: 400; loss: 0.5601604580879211; accuracy: 0.796875
epoch: 2 | batch_num: 600; loss: 0.6656577587127686; accuracy: 0.796875
epoch: 2 | batch_num: 800; loss: 0.8908078074455261; accuracy: 0.765625

epoch: 2 | Minimum loss of model during training is batch_num: 895; loss: 0.3273125886917114; accuarcy: 0.90625
epoch: 2 | test loss: 0.5264596343040466; accuarcy: 0.7919
==================== epoch 3


epoch: 3 | batch_num: 0; loss: 0.6894076466560364; accuracy: 0.8125
epoch: 3 | batch_num: 200; loss: 0.5760871171951294; accuracy: 0.796875
epoch: 3 | batch_num: 400; loss: 0.590149998664856; accuracy: 0.796875
epoch: 3 | batch_num: 600; loss: 0.6288356184959412; accuracy: 0.78125
epoch: 3 | batch_num: 800; loss: 0.7075309157371521; accuracy: 0.75

epoch: 3 | Minimum loss of model during training is batch_num: 268; loss: 0.31884223222732544; accuarcy: 0.90625
epoch: 3 | test loss: 0.5017235279083252; accuarcy: 0.8114
==================== epoch 4


epoch: 4 | batch_num: 0; loss: 0.5278751850128174; accuracy: 0.8125
epoch: 4 | batch_num: 200; loss: 0.5037315487861633; accuracy: 0.84375
epoch: 4 | batch_num: 400; loss: 0.45365625619888306; accuracy: 0.859375
epoch: 4 | batch_num: 600; loss: 0.8407962918281555; accuracy: 0.71875
epoch: 4 | batch_num: 800; loss: 0.5849816799163818; accuracy: 0.78125

epoch: 4 | Minimum loss of model during training is batch_num: 707; loss: 0.21515798568725586; accuarcy: 0.953125
epoch: 4 | test loss: 0.4812643527984619; accuarcy: 0.8152
==================== epoch 5


epoch: 5 | batch_num: 0; loss: 0.6889262795448303; accuracy: 0.734375
epoch: 5 | batch_num: 200; loss: 0.7072533369064331; accuracy: 0.703125
epoch: 5 | batch_num: 400; loss: 0.4076870083808899; accuracy: 0.875
epoch: 5 | batch_num: 600; loss: 0.4611160159111023; accuracy: 0.859375
epoch: 5 | batch_num: 800; loss: 0.7877468466758728; accuracy: 0.796875

epoch: 5 | Minimum loss of model during training is batch_num: 606; loss: 0.2451651692390442; accuarcy: 0.921875
epoch: 5 | test loss: 0.4582318961620331; accuarcy: 0.8327


책에서는 40번의 epoch으로 학습을 진행하고, test set에 대한 accuracy 점수는 99%에 달한다.

이전 4장에서 만들었던 DNN과 비교하였을때, 64% -> 84%로 정확도가 20% 증가하였다. (물론, CNN에서는 epoch을 6번을 순회하면서 진행했지만 말이다.)

## 5.3. ResNet으로 컬러 데이터셋에 적용하기

기본적인 CNN 신경망에서 더 좋은 성능을 내기 위해 신경망 구조에 변형을 주는 많은 시도들이 이어졌다. 그 중 2015년에 등장한 ResNet에 대해서 알아보고 구현해보도록 하자.

<img src="https://blog.kakaocdn.net/dn/bFPOry/btqzR2En9ry/2DTETgT1BkCrW74hKQCsrk/img.png"/>

ResNet에서 가장 핵심적인 요소는 위의 그림과 같은 Residential Block의 등장이다. 딥러닝은 학습을 진행하면 할 수록 성능이 나빠질 수 있다. 왜냐하면 과거에 학습한 가중치들이 계층을 거칠 수록 잊혀지는 문제가 발생하기 때문이다. 따라서, ResNet에서는 위의 그림과 같이 특정 컨볼루션 레이어의 출력에 이전 컨볼루션 레이어의 입력을 더함으로써 특징이 유실되지 않도록 하는 것이다. 이 때문에 CNN 모델의 레이어를 더 깊게 쌓을 수 있으며, 신경망이 깊어짐에 따라 문제를 더 작은 단위로 분해하기 때문에 학습효율이 좋아진다.

우리는 ResNet을 위해서 기존에 사용했던 1 channel인 Fashion MNIST 데이터셋에서 확장하여 3 channel image인 CIFAR-10 데이터셋을 활용한다.

ResNet 모델을 구현하는데 있어서 더 간편하게 구현하기 위해서 Basic Block 이라는 새로운 파이토치 모듈을 선언하여 사용하자.

In [17]:
EPOCHS = 300
BATCH_SIZE = 32

In [18]:
train_transform = transforms.Compose([
#     transforms.ToPILImage(),
    transforms.RandomCrop(32,padding=1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

test_transform = transforms.Compose([
#     transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [19]:
train_dataset = datasets.CIFAR10(
    root = './data/',
    train = True,
#     download = True,
    download = False,
    transform = train_transform
)

test_dataset = datasets.CIFAR10(
    root = './data/',
    train = False,
#     download = True,
    download = False,
    transform = test_transform
)

In [20]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False
)

### Batch Normalization

학습과정에서 학습률이 너무 높으면 기울기가 소실 또는 발산하는 증상을 예방하기 위해서 학습 과정을 안정화하는 방법. 각 계층에 들어가는 입력을 평균과 분산으로 정규화함으로써 학습을 효율적으로 만들어준다. 이는 자체적으로 정규화를 수행하기 때문에 드롭아웃과 같은 효과를 낸다.

* Why does Batch Norm Work?

    Video link: https://www.youtube.com/watch?v=nUUqwaxLnWs
    
    Review link: <a href="./Why does Batch Norm Work.md">move to markdown</a>

In [21]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn1(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        
        return out

In [22]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3,16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64,num_classes)
    
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
    
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out,8)
        out = out.view(out.size(0),-1)
        out = self.linear(out)
        
        return out

In [23]:
model = ResNet().to(DEVICE) if USE_CUDA else ResNet()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [24]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [25]:
for epoch in range(EPOCHS):
    scheduler.step()
    model = train(model, train_loader, optimizer, epoch,batch_size=BATCH_SIZE)
    evaluate(model, test_loader, epoch, BATCH_SIZE)
    
    if epoch > 1:
        break

C:\Users\kwon2\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


epoch: 0 | batch_num: 0; loss: 2.3470845222473145; accuracy: 0.15625
epoch: 0 | batch_num: 200; loss: 1.951581358909607; accuracy: 0.1875
epoch: 0 | batch_num: 400; loss: 1.5447102785110474; accuracy: 0.46875
epoch: 0 | batch_num: 600; loss: 1.1088913679122925; accuracy: 0.625
epoch: 0 | batch_num: 800; loss: 1.1309459209442139; accuracy: 0.59375
epoch: 0 | batch_num: 1000; loss: 1.3013243675231934; accuracy: 0.46875
epoch: 0 | batch_num: 1200; loss: 0.9972138404846191; accuracy: 0.5
epoch: 0 | batch_num: 1400; loss: 1.005329966545105; accuracy: 0.625

epoch: 0 | Minimum loss of model during training is batch_num: 1458; loss: 0.6959466338157654; accuarcy: 0.6875
epoch: 0 | test loss: 3.389369487762451; accuarcy: 0.2135


epoch: 1 | batch_num: 0; loss: 1.1485624313354492; accuracy: 0.6875
epoch: 1 | batch_num: 200; loss: 1.1316574811935425; accuracy: 0.5
epoch: 1 | batch_num: 400; loss: 0.8894685506820679; accuracy: 0.625
epoch: 1 | batch_num: 600; loss: 1.146327018737793; accuracy: 0.65625
epoch: 1 | batch_num: 800; loss: 1.0024006366729736; accuracy: 0.625
epoch: 1 | batch_num: 1000; loss: 0.7219847440719604; accuracy: 0.71875
epoch: 1 | batch_num: 1200; loss: 1.1257085800170898; accuracy: 0.6875
epoch: 1 | batch_num: 1400; loss: 1.23028564453125; accuracy: 0.625

epoch: 1 | Minimum loss of model during training is batch_num: 1503; loss: 0.43501773476600647; accuarcy: 0.875
epoch: 1 | test loss: 2.9705936908721924; accuarcy: 0.2798


epoch: 2 | batch_num: 0; loss: 0.6462538242340088; accuracy: 0.8125
epoch: 2 | batch_num: 200; loss: 0.9362425208091736; accuracy: 0.65625
epoch: 2 | batch_num: 400; loss: 1.403009057044983; accuracy: 0.59375
epoch: 2 | batch_num: 600; loss: 0.4538954496383667; accuracy: 0.8125
epoch: 2 | batch_num: 800; loss: 0.8163824677467346; accuracy: 0.6875
epoch: 2 | batch_num: 1000; loss: 0.7630157470703125; accuracy: 0.71875
epoch: 2 | batch_num: 1200; loss: 0.872978687286377; accuracy: 0.65625
epoch: 2 | batch_num: 1400; loss: 0.5652733445167542; accuracy: 0.8125

epoch: 2 | Minimum loss of model during training is batch_num: 223; loss: 0.34368792176246643; accuarcy: 0.875
epoch: 2 | test loss: 4.4041571617126465; accuarcy: 0.182


역설적이게도 책의 예제 코드와는 달리 test값에 대한 정확도와 loss가 굉장히 낮다. 이는 왜인지 모르겠지만, 과대적합이 발생하여 test값에 대한 예측이 올바르게 이루어지지 않는 듯하다. 

책에서는 해당 데이터셋이 성능을 평가하기에 적합하지 않다고하며, train셋의 데이터 크기를 늘려주게 되면 원하는 수준의 test 예측결과를 얻을 수 있다고 한다.